# ABS Quarterly Dwelling Stock 6432

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# local imports
import abs_data_capture as adc
import plotting as p

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [3]:
def get_data() -> tuple[dict[str, pd.DataFrame], pd.DataFrame, str]:
    """Capture relevant ABS data, set-up plotting output directories"""

    landing_page_ = adc.AbsLandingPage(
        theme="economy",
        parent_topic="price-indexes-and-inflation",
        topic="total-value-dwellings",
    )
    abs_dict_ = adc.get_abs_data(landing_page_)
    source_, chart_dir, _cat_id, meta_ = adc.get_fs_constants(abs_dict_, landing_page_)

    # plotting set-up
    p.clear_chart_dir(chart_dir)
    p.set_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")

    return abs_dict_, meta_, source_

In [4]:
abs_dict, meta, source = get_data()

# let's get a quick look at the data
print(abs_dict["1"].tail().index)

A little unexpected: We need to fake up a zip file
PeriodIndex(['2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4'], dtype='period[Q-DEC]', name='Series ID')


## Plot the data

In [5]:
def headline() -> None:
    """Headline charts"""

    plotable = [
        "Value of dwelling stock; Owned by Households ;  Australia ;",
        "Value of dwelling stock; Owned by All Sectors ;  Australia ;",
        "Value of dwelling stock; Owned by Non-Households ;  Australia ;",
        "Mean price of residential dwellings ;  Australia ;",
        "Number of residential dwellings ;  Australia ;",
    ]
    data = abs_dict["1"]
    for item in plotable:
        search = {item: adc.metacol.did}
        series_id, units = adc.find_id(meta, search)
        series, units = p.recalibrate_series(data[series_id], units, verbose=True)
        stype = meta[meta[adc.metacol.id] == series_id][adc.metacol.stype].values[0]

        ax = series.plot()
        p.finalise_plot(
            ax,
            title=item.rsplit(";", maxsplit=2)[0],
            ylabel=units,
            rfooter=source,
            lfooter=f"Australia. {stype} series. ",
            show=SHOW,
        )

In [6]:
headline()

recalibrate(): No adjustments needed


## Calculate and plot population per dwelling

In [7]:
def get_population() -> tuple[pd.Series, str]:
    """Get latest population estimates from national accounts."""

    landing_page = adc.AbsLandingPage(
        theme="economy",
        parent_topic="national-accounts",
        topic="australian-national-accounts-national-income-expenditure-and-product",
    )
    gdp_dict = adc.get_abs_data(landing_page)
    _, _, cat_id, gdp_meta = adc.get_fs_constants(gdp_dict, landing_page)

    gdp_did = "Gross domestic product"
    gdp_pc_did = "GDP per capita"
    table = "1"
    data = pd.DataFrame()
    for item in gdp_did, gdp_pc_did:
        search = {
            item: adc.metacol.did,
            "Chain volume measures": adc.metacol.did,
            "$": adc.metacol.unit,
            "Original": adc.metacol.stype,
            table: adc.metacol.table,
        }
        series_id, _unit = adc.find_id(gdp_meta, search, verbose=False)
        data[item] = gdp_dict[table][series_id]
    population = data[data.columns[0]] / data[data.columns[1]] * 1_000_000

    return population, cat_id

In [8]:
def pop_per_dwelling() -> None:
    """Plot population per dwelling."""

    population, cat_gdp = get_population()

    search = {"Number of residential dwellings ;  Australia ;": adc.metacol.did}
    series_id, units = adc.find_id(meta, search)
    assert units == "Thousands"
    dwellings = abs_dict["1"][series_id] * 1_000
    pop_per_dwellings = (population / dwellings).dropna()

    ax = pop_per_dwellings.plot()
    p.finalise_plot(
        ax,
        title="Implicit population per dwelling",
        ylabel="Persons",
        rfooter=f"{source} {cat_gdp}",
        lfooter="Australia. Original series. ",
        show=SHOW,
    )

In [9]:
pop_per_dwelling()

## Calculate and plot mean dwelling value per FT annual ordinary earnings 

In [10]:
def get_earnings() -> tuple[pd.Series, str]:
    """Return Average Annual FT Adult Ordinary Time Earnings for Persons.
    Note: This data is published biannually."""

    landing_page = adc.AbsLandingPage(
        theme="labour",
        parent_topic="earnings-and-working-conditions",
        topic="average-weekly-earnings-australia",
    )
    awe_dict = adc.get_abs_data(landing_page)
    _, _, cat_id, awe_meta = adc.get_fs_constants(awe_dict, landing_page)

    table = '3'  # Original series
    awe_did = "Earnings; Persons; Full Time; Adult; Ordinary time earnings ;"
    search = {
        awe_did: adc.metacol.did,
        table: adc.metacol.table,
     }
    series_id, _unit = adc.find_id(awe_meta, search, exact=True, verbose=False)
    annual_ft_earnings = awe_dict[table][series_id] * 365.25 / 7
    return annual_ft_earnings, cat_id

In [11]:
def mean_dwelling_value_per_earnings() -> None:
    """Plot mean dwelling value per average annual FT ordinary time earnings."""

    earnings, cat_awe = get_earnings()
    earnings = earnings.to_timestamp(how='end').to_period(freq='Q-DEC')

    search = {"Mean price of residential dwellings ;  Australia ;": adc.metacol.did}
    series_id, units = adc.find_id(meta, search)
    assert units.strip() == '$ Thousand'
    mean_value = abs_dict["1"][series_id] * 1_000
    value_per_earnings = (mean_value / earnings).dropna()

    ax = value_per_earnings.plot()
    p.finalise_plot(
        ax,
        title="Mean dwelling value / Annual ave FT ordinary earnings",
        ylabel="Multiples",
        rfooter=f"{source} {cat_awe}",
        lfooter="Australia. Original series. ",
        show=SHOW,
    )

In [12]:
mean_dwelling_value_per_earnings()

Caution: Could not find the 'Index' sheet in 63020do015_20234.xlsx. File not included
Caution: Could not find the 'Index' sheet in 63020do016_20234.xlsx. File not included
Caution: Could not find the 'Index' sheet in 63020do017_20234.xlsx. File not included


## Finished

In [13]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Mar 15 2024 11:10:26

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

pandas    : 2.2.1
matplotlib: 3.8.3

Watermark: 2.4.3



In [14]:
print("Finished")

Finished
